In [93]:
import os
import io
import csv
import pandas as pd
import numpy as np
from tabulate import tabulate
import logging
logging.getLogger().setLevel(logging.INFO)

import psycopg2
conn = psycopg2.connect(user='airflow', password='Xypherium-0',
                        dbname='jpstat',
                        host='35.224.240.50')

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [3]:
path = os.path.join('extracts', 'monthly_reports')

In [ ]:
def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = dict.fromkeys(files)
        parent = reduce(dict.get, folders[:-1], dir)
        parent[folders[-1]] = subdir
    return dir

In [5]:
filepathlist = []
for dirpath, subdir, files in os.walk(path):
    for filename in files:
        if filename.endswith('.xls'):
            filepathlist.append(os.path.join(dirpath, filename))

In [97]:
%%time
tableshape_list = []
df_dict = {}
for filepath in filepathlist:
    sheets = pd.ExcelFile(filepath).sheet_names
    
    df = pd.read_excel(filepath, sheet_name=len(sheets)-1)
    filepath_list = filepath.split('\\')
    description = [filepath_list[2] + '-' + filepath_list[3], filepath_list[4], df.shape]
    
    if len(sheets) > 1:
        print(filepath_list[2] + '-' + filepath_list[3] + ' ' + filepath_list[4] + ' has ' + str(len(sheets)) + ' sheets.')
    
    tableshape_list.append(description)

2013-07Jul 1.xls has 2 sheets.
2013-07Jul 3-3.xls has 2 sheets.
2013-08Aug 1.xls has 2 sheets.
2013-08Aug 3-3.xls has 2 sheets.
2013-09Sep 1.xls has 2 sheets.
2013-09Sep 3-3.xls has 2 sheets.
2013-10Oct 1.xls has 2 sheets.
2013-10Oct 3-3.xls has 2 sheets.
2013-11Nov 1.xls has 2 sheets.
2013-11Nov 3-3.xls has 2 sheets.
2013-12Dec 1.xls has 2 sheets.
2013-12Dec 3-3.xls has 2 sheets.
2014-01Jan 1.xls has 2 sheets.
2014-01Jan 3-3.xls has 2 sheets.
2014-02Feb 1.xls has 2 sheets.
2014-02Feb 3-3.xls has 2 sheets.
2014-03Mar 1.xls has 2 sheets.
2014-03Mar 3-3.xls has 2 sheets.
2014-04Apr 1.xls has 2 sheets.
2014-04Apr 3-3.xls has 2 sheets.
2014-05May 1.xls has 2 sheets.
2014-05May 3-3.xls has 2 sheets.
2014-06Jun 1.xls has 2 sheets.
2014-06Jun 3-3.xls has 2 sheets.
2014-07Jul 1.xls has 2 sheets.
2014-07Jul 3-3.xls has 2 sheets.
2014-08Aug 1.xls has 2 sheets.
2014-08Aug 3-3.xls has 2 sheets.
2014-09Sep 1.xls has 2 sheets.
2014-09Sep 3-3.xls has 2 sheets.
2014-10Oct 1.xls has 2 sheets.
2014-10Oc

In [104]:
tableshape_df = pd.DataFrame(tableshape_list, columns=['year-month', 'file', 'shape'])

In [105]:
tableshape_df.info

<bound method DataFrame.info of      year-month     file       shape
0    2006-01Jan  1.xls    (88, 13)  
1    2006-01Jan  2.xls    (1285, 12)
2    2006-02Feb  1.xls    (88, 13)  
3    2006-02Feb  2.xls    (1285, 12)
4    2006-03Mar  1.xls    (88, 13)  
5    2006-03Mar  2.xls    (1285, 12)
6    2006-04Apr  1.xls    (89, 13)  
7    2006-04Apr  2.xls    (1365, 12)
8    2006-05May  1.xls    (89, 13)  
9    2006-05May  2.xls    (1365, 12)
10   2006-06Jun  1.xls    (89, 13)  
11   2006-06Jun  2.xls    (1365, 12)
12   2006-07Jul  1.xls    (89, 13)  
13   2006-07Jul  2.xls    (1365, 12)
14   2006-08Aug  1.xls    (89, 13)  
15   2006-08Aug  2.xls    (1365, 12)
16   2006-09Sep  1.xls    (89, 13)  
17   2006-09Sep  2.xls    (1365, 12)
18   2006-10Oct  1.xls    (86, 18)  
19   2006-10Oct  2.xls    (1362, 18)
20   2006-11Nov  1.xls    (86, 18)  
21   2006-11Nov  2.xls    (1362, 18)
22   2006-12Dec  1.xls    (86, 18)  
23   2006-12Dec  2.xls    (1362, 18)
24   2007-01Jan  1.xls    (86, 18)  
25   2

In [106]:
tableshape_agg_df = tableshape_df.groupby(['file', 'shape'])['year-month'].apply(list).reset_index()

In [107]:
tableshape_agg_df['count'] = [len(x) for x in tableshape_agg_df['year-month']]
[x.sort for x in tableshape_agg_df['year-month']]
tableshape_agg_df.sort_values(by='year-month')
tableshape_agg_df

,file,shape,year-month,count
0,1.xls,"(86, 18)","[2006-10Oct, 2006-11Nov, 2006-12Dec, 2007-01Jan, 2007-02Feb, 2007-03Mar]",6
1,1.xls,"(88, 13)","[2006-01Jan, 2006-02Feb, 2006-03Mar]",3
2,1.xls,"(88, 18)","[2007-04Apr, 2007-05May, 2007-06Jun, 2007-07Jul, 2007-08Aug, 2007-09Sep, 2007-10Oct, 2007-11Nov, 2007-12Dec]",9
3,1.xls,"(89, 13)","[2006-04Apr, 2006-05May, 2006-06Jun, 2006-07Jul, 2006-08Aug, 2006-09Sep]",6
4,1.xls,"(91, 13)","[2008-01Jan, 2008-02Feb, 2008-03Mar, 2008-04Apr, 2008-05May, 2008-06Jun, 2008-07Jul, 2008-08Aug, 2008-09Sep, 2008-10Oct, 2008-11Nov, 2008-12Dec, 2009-01Jan, 2009-02Feb, 2009-03Mar, 2010-01Jan, 2010-02Feb, 2010-03Mar]",18
5,1.xls,"(92, 13)","[2009-04Apr, 2009-05May, 2009-06Jun, 2009-07Jul, 2009-08Aug, 2009-09Sep, 2009-10Oct, 2009-11Nov, 2009-12Dec, 2010-04Apr, 2010-05May, 2010-06Jun, 2010-07Jul, 2010-08Aug, 2010-09Sep, 2010-10Oct, 2010-11Nov, 2010-12Dec, 2011-01Jan, 2011-02Feb, 2011-03Mar, 2011-04Apr, 2011-05May, 2011-06Jun, 2011-07Jul, 2011-08Aug, 2011-09Sep, 2011-10Oct, 2011-11Nov, 2011-12Dec, 2012-01Jan, 2012-02Feb, 2012-03Mar]",33
6,1.xls,"(93, 13)","[2012-04Apr, 2012-05May, 2012-06Jun, 2012-07Jul, 2012-08Aug, 2012-09Sep, 2012-10Oct, 2012-11Nov, 2012-12Dec, 2013-01Jan, 2013-02Feb, 2013-03Mar, 2013-04Apr, 2013-05May, 2013-06Jun, 2013-07Jul, 2013-08Aug, 2013-09Sep, 2013-10Oct, 2013-11Nov, 2013-12Dec, 2014-01Jan, 2014-02Feb, 2014-03Mar, 2014-04Apr, 2014-05May, 2014-06Jun, 2014-07Jul, 2014-08Aug, 2014-09Sep, 2014-10Oct, 2014-11Nov, 2014-12Dec, 2015-01Jan, 2015-02Feb, 2015-03Mar, 2015-04Apr, 2015-05May, 2015-06Jun, 2015-07Jul, 2015-08Aug, 2015-09Sep, 2015-10Oct, 2015-11Nov, 2015-12Dec, 2016-01Jan, 2016-02Feb, 2016-03Mar, 2016-04Apr, 2016-05May, 2016-06Jun, 2016-07Jul, 2016-08Aug, 2016-09Sep, 2016-10Oct, 2016-11Nov, 2016-12Dec, 2017-01Jan, 2017-02Feb, 2017-03Mar, 2017-04Apr, 2017-05May, 2017-06Jun, 2017-07Jul, 2017-08Aug, 2017-09Sep, 2017-10Oct, 2017-11Nov, 2017-12Dec]",69
7,1.xls,"(105, 13)","[2018-01Jan, 2018-02Feb, 2018-03Mar, 2018-04Apr, 2018-05May, 2018-06Jun]",6
8,2.xls,"(1285, 12)","[2006-01Jan, 2006-02Feb, 2006-03Mar]",3
9,2.xls,"(1362, 18)","[2006-10Oct, 2006-11Nov, 2006-12Dec, 2007-01Jan, 2007-02Feb, 2007-03Mar]",6


In [6]:
table_1_list= [x for x in filepathlist if x.split('\\')[-1] == '1.xls']
table3_1_list = [x for x in filepathlist if '3-1' in x]
table3_2_list = [x for x in filepathlist if '3-2' in x]

In [21]:
def combine_clean_1xls_tables(table_excel_list):
    table_df_list = []

    for filepath in table_excel_list:
        filepath_parts = filepath.split('\\')
        year = filepath_parts[2]
        month = filepath_parts[3][:2]

        sheets = pd.ExcelFile(filepath).sheet_names
        df = pd.read_excel(filepath, sheet_name=len(sheets)-1)

        messy_df = df.copy()
        messy_df.reset_index(inplace=True)
        
        cornerstone = '全国'
        cornerstone_col_idx = [messy_df.columns.get_loc(x) for x in messy_df.columns if cornerstone in messy_df[x].values][0]
        cornerstone_col = messy_df.iloc[:,cornerstone_col_idx]
        cornerstone_row_idx = int(cornerstone_col.index[cornerstone_col == cornerstone][0])
        messy_df = messy_df.iloc[:,cornerstone_col_idx:cornerstone_col_idx+6]

        messy_df.columns = ['Prefecture Num', 'Prefecture', 'Prefecture_English', 'Both', 'Male', 'Female']
        messy_df.dropna(axis=0, subset=['Prefecture Num', 'Both', 'Male', 'Female'], how='any', inplace=True)
        
        #using the fact that np.nan != np.nan
        messy_df['Prefecture'] = messy_df.apply(lambda x: x['Prefecture Num'] if x['Prefecture']!=x['Prefecture'] else x['Prefecture'], axis=1)
        messy_df['Prefecture Num'] = messy_df.apply(lambda x: '00' if x['Prefecture Num']=='全国' else x['Prefecture Num'], axis=1)
        messy_df['Prefecture Num'] = messy_df.apply(lambda x: '00 000' if '計' in x['Prefecture Num'] else x['Prefecture Num'], axis=1)
        ku_codes = {
            '東京圏': '100',
            '名古屋圏': '101',
            '大阪圏': '102'
        }
        messy_df['Prefecture Num'] = messy_df.apply(lambda x: ku_codes.get(x['Prefecture Num']) if x['Prefecture Num'] in ku_codes else x['Prefecture Num'], axis=1)
        messy_df['Prefecture_English'] = messy_df.apply(lambda x: x['Prefecture_English'].rstrip(' 3)'), axis=1)
        messy_df['Prefecture_English'] = messy_df.apply(lambda x: x['Prefecture_English'] + ' cities' if 'major' in x['Prefecture_English'] else x['Prefecture_English'], axis=1)
        
        messy_df['Year'] = str(year)
        messy_df['Month'] = str(month)
        messy_df['Age bucket'] = 'Total'

        messy_df = messy_df.reindex(columns=['Prefecture Num', 'Prefecture', 'Prefecture_English', 'Year', 'Month', 'Age bucket', 'Both', 'Male', 'Female'])
        
        if False not in messy_df.iloc[:,7:10].applymap(np.isreal).values.flatten():
            table_df_list.append(messy_df)
        else:
            print('Some non-numeric value found in internal migration counts.')
            print(messy_df.iloc[:,7:10].head(5))

    combined_df = pd.concat(table_df_list).reset_index(drop=True)
    if '大阪圏' in combined_df['Prefecture Num']: print('matched multiple kanji string!')
    return combined_df

In [8]:
def combine_clean_3xls_tables(table_excel_list):
    #Always take the last sheet: assume that in the absense of separate sheets, the numbers are for Japanese citizens only
    table_df_list = []

    for filepath in table_excel_list:
        filepath_parts = filepath.split('\\')
        year = filepath_parts[2]
        month = filepath_parts[3][:2]

        sheets = pd.ExcelFile(filepath).sheet_names
        df = pd.read_excel(filepath, sheet_name=len(sheets)-1)

        messy_df = df.copy()
        messy_df.reset_index(inplace=True)
        messy_df.columns = [x.replace('～', '-') for x in messy_df.iloc[13].replace({
                                                    'Prefectures': 'Prefecture Num',
                                                    '総 数 1)': 'Total',
                                                    '0～4歳': '0～4',
                                                    '90歳以上': '90 & above'
                                                    }).fillna('Prefecture')]
        messy_df.iloc[18,10] = messy_df.iloc[18,8]
        messy_df = messy_df.iloc[18:66,8:]
        messy_df.reset_index(drop=True, inplace=True)
        messy_df.dropna(axis=1, inplace=True)
        messy_df.loc[0,'Prefecture Num'] = '00'

        both_df = messy_df.iloc[:,:22]
        male_df = messy_df.iloc[:,22:44]
        female_df = messy_df.iloc[:,44:66]

        both_df['Gender'] = 'Both'
        male_df['Gender'] = 'Male'
        female_df['Gender'] = 'Female'

        this_month_df = pd.concat([both_df, male_df, female_df])
        this_month_df['Year'] = str(year)
        this_month_df['Month'] = str(month)

        if False not in this_month_df.iloc[:,2:22].applymap(np.isreal).values.flatten():
            table_df_list.append(this_month_df)
        else:
            print('Some non-numeric value found in in/out migration counts.')
    
    combined_df = pd.concat(table_df_list).reset_index(drop=True)
    return combined_df

In [22]:
%%time
in_migrants_age_df = combine_clean_3xls_tables(table3_1_list)
out_migrants_age_df = combine_clean_3xls_tables(table3_2_list)
internal_migrants_df = combine_clean_1xls_tables(table_1_list)

C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Wall time: 27.1 s


In [68]:
jp2en_df = internal_migrants_df[['Prefecture', 'Prefecture_English']].drop_duplicates()
jp2en_df.set_index('Prefecture', inplace=True)
jp2en_dict = jp2en_df['Prefecture_English'].to_dict()

In [71]:
jp2en_dict.get('北海道')

' Hokkaido'

In [52]:
internal_migrants_df.tail(60)

,Prefecture Num,Prefecture,Prefecture_English,Year,Month,Age bucket,Both,Male,Female
10260,13,東京都,Tokyo-to,2018,06,Total,28529,14549,13980
10261,14,神奈川県,Kanagawa-ken,2018,06,Total,15365,7878,7487
10262,15,新潟県,Niigata-ken,2018,06,Total,2196,1075,1121
10263,16,富山県,Toyama-ken,2018,06,Total,657,316,341
10264,17,石川県,Ishikawa-ken,2018,06,Total,990,474,516
10265,18,福井県,Fukui-ken,2018,06,Total,569,278,291
10266,19,山梨県,Yamanashi-ken,2018,06,Total,974,457,517
10267,20,長野県,Nagano-ken,2018,06,Total,2044,1007,1037
10268,21,岐阜県,Gifu-ken,2018,06,Total,1643,844,799
10269,22,静岡県,Shizuoka-ken,2018,06,Total,4395,2275,2120


In [49]:
keys_1xls = ['Both', 'Male', 'Female']
internal_melted_df = pd.melt(internal_migrants_df, 
                             id_vars=['Prefecture Num', 'Prefecture', 'Prefecture_English', 'Year', 'Month', 'Age bucket'], 
                             value_name='Internal migrants', 
                             value_vars=keys_1xls, 
                             var_name='Gender')

In [50]:
len(internal_melted_df)

30960

In [51]:
out_migrants_age_df.head(20)

,Prefecture Num,Prefecture,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90 & above,Gender,Year,Month
0,00,全国,126018,7924,3182,1637,3610,20043,26958,20002,13587,7826,4881,3811,3580,3247,1722,1098,946,864,711,387,Both,2010,01
1,01,北海道,2561,136,77,45,103,473,441,365,296,186,112,99,98,47,27,18,11,13,9,5,Both,2010,01
2,02,青森県,1119,66,38,30,34,277,186,152,103,51,47,39,37,22,9,9,4,5,8,2,Both,2010,01
3,03,岩手県,992,74,31,10,37,201,167,155,83,50,59,39,31,23,11,5,6,5,2,3,Both,2010,01
4,04,宮城県,2085,160,59,28,66,372,422,298,242,123,88,74,54,38,17,11,13,9,8,3,Both,2010,01
5,05,秋田県,753,50,24,11,25,176,121,98,71,39,27,23,25,17,14,4,10,11,3,4,Both,2010,01
6,06,山形県,736,43,27,11,35,165,139,99,70,32,22,27,17,21,9,7,3,1,6,2,Both,2010,01
7,07,福島県,1386,87,25,17,48,336,253,186,128,87,57,44,32,23,12,12,7,16,11,5,Both,2010,01
8,08,茨城県,2564,177,78,39,74,409,518,419,289,141,90,73,76,62,29,31,17,13,16,13,Both,2010,01
9,09,栃木県,1686,102,45,32,49,304,355,226,166,120,60,56,51,49,26,14,12,11,5,3,Both,2010,01


In [53]:
keys_3xls = ['Total', '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90 & above']
in_melted_df = pd.melt(in_migrants_age_df, 
                       id_vars=['Prefecture Num', 'Prefecture', 'Gender', 'Year', 'Month'], 
                       value_name='Inmigrants', 
                       value_vars=keys_3xls, 
                       var_name='Age bucket')
out_melted_df = pd.melt(out_migrants_age_df, 
                        id_vars=['Prefecture Num', 'Prefecture', 'Gender', 'Year', 'Month'], 
                        value_name='Outmigrants', 
                        value_vars=keys_3xls, 
                        var_name='Age bucket')

In [54]:
combined_melted_df = pd.merge(in_melted_df, out_melted_df, how='left',
                              left_on=list(in_melted_df.columns).remove('Inmigrants'), 
                              right_on=list(out_melted_df.columns).remove('Outmigrants'))

In [55]:
len(combined_melted_df)

293760

In [56]:
out_melted_df.shape[0] == combined_melted_df.shape[0]

True

In [57]:
shared_columns = [x for x in list(internal_melted_df.columns) if x in list(combined_melted_df.columns)]
final_combined_melted_df = pd.merge(combined_melted_df, internal_melted_df, how='outer',
                              left_on=shared_columns, 
                              right_on=shared_columns)

In [58]:
len(final_combined_melted_df)

310032

In [73]:
final_combined_melted_df['Prefecture_English'] = final_combined_melted_df.apply(lambda x: jp2en_dict.get(x['Prefecture']), axis=1)

In [94]:
abcdf = pd.DataFrame(final_combined_melted_df.isnull().sum())
logging.info(tabulate(abcdf, headers='keys', tablefmt='psql'))

INFO:root:+--------------------+--------+
|                    |      0 |
|--------------------+--------|
| Prefecture Num     |      0 |
| Prefecture         |      0 |
| Gender             |      0 |
| Year               |      0 |
| Month              |      0 |
| Age bucket         |      0 |
| Inmigrants         |  16272 |
| Outmigrants        |  16272 |
| Prefecture_English |      0 |
| Internal migrants  | 279072 |
+--------------------+--------+


About 30 months of major city and metropolitan area internal migration data for which there is no corresponding in/out migration data.

Internal migration data only available for total population of each gender, not each age bracket.

In [78]:
final_combined_melted_df.tail(30)

,Prefecture Num,Prefecture,Gender,Year,Month,Age bucket,Inmigrants,Outmigrants,Prefecture_English,Internal migrants
310002,33 100,岡山市,Female,2018,05,Total,NaN,NaN,Okayama-shi,419
310003,34 100,広島市,Female,2018,05,Total,NaN,NaN,Hiroshima-shi,1028
310004,40 100,北九州市,Female,2018,05,Total,NaN,NaN,Kitakyushu-shi,652
310005,40 130,福岡市,Female,2018,05,Total,NaN,NaN,Fukuoka-shi,1599
310006,43 100,熊本市,Female,2018,05,Total,NaN,NaN,Kumamoto-shi,755
310007,100,東京圏,Female,2018,06,Total,NaN,NaN,Tokyo Area,44952
310008,101,名古屋圏,Female,2018,06,Total,NaN,NaN,Nagoya Area,8681
310009,102,大阪圏,Female,2018,06,Total,NaN,NaN,Osaka Area,16957
310010,00 000,21 大 都 市 計,Female,2018,06,Total,NaN,NaN,Total of 21 major cities,27215
310011,01 100,札幌市,Female,2018,06,Total,NaN,NaN,Sapporo-shi,2282


In [ ]:
def dataframe_to_postgres(df, tablename):
    cur = conn.cursor()
    textstream = io.StringIO()
    upload_df = df.copy()
    #upload_df['excel_description'].replace(['\n', '\t'], '', regex=True, inplace=True)

    upload_df.to_csv(textstream, sep='\t', header=False, index=False, quoting=csv.QUOTE_NONE)
    textstream.seek(0) 
    cur.copy_from(textstream, tablename, null="") # null values become ''
    conn.commit()

In [96]:
table_2_list= [x for x in filepathlist if x.split('/')[-1] == '2.xls']